In [1]:
import numpy as np
import pandas as pd
import featuretools as ft

operational_settings = ['operational_setting_{}'.format(i + 1) for i in range (3)]
sensor_columns = ['sensor_measurement_{}'.format(i + 1) for i in range(26)]
cols = ['engine_no', 'time_in_cycles'] + operational_settings + sensor_columns

data = pd.read_csv('data/RUL_train.txt', sep=' ', header=-1, names=cols)

data = data.drop(cols[-5:], axis=1)
data['index'] = data.index
data.index = data['index']
data.head()

,engine_no,time_in_cycles,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,index
index,,,,,,,,,,,,,,,,,,,,,
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670,0
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552,1
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213,2
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176,3
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754,4


In [2]:
from random import randint
data['time'] = pd.date_range('1/1/2000', periods=data.shape[0], freq='s')

gb = data.groupby(['engine_no'])
labels = []


for engine_no_df in gb:
    instances = engine_no_df[1].shape[0]
    label = [instances - i - 1 for i in range(instances)]
    labels += label
data['label'] = labels

def new_labels(data, labels):
    ct_ids = []
    ct_times = []
    ct_labels = []
    data['label'] = labels
    gb = data.groupby(['engine_no'])
    for engine_no_df in gb:
        instances = engine_no_df[1].shape[0]
        r = randint(5, instances - 1)
        ct_ids.append(engine_no_df[1].iloc[r,:]['engine_no'])
        ct_times.append(engine_no_df[1].iloc[r,:]['time'])
        ct_labels.append(engine_no_df[1].iloc[r,:]['label'])
    ct = pd.DataFrame({'engine_no': ct_ids,
                       'cutoff_time': ct_times,
                       'label': ct_labels})
    ct = ct[['engine_no', 'cutoff_time', 'label']]
    ct.index = ct['engine_no']
    return ct
data.tail()

,engine_no,time_in_cycles,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,index,time,label
index,,,,,,,,,,,,,,,,,,,,,
61244,249,251,9.9998,0.2500,100.0,489.05,605.33,1516.36,1315.28,10.52,...,8.4541,0.03,372,2319,100.0,29.11,17.5234,61244,2000-01-01 17:00:44,4
61245,249,252,0.0028,0.0015,100.0,518.67,643.42,1598.92,1426.77,14.62,...,8.2221,0.03,396,2388,100.0,39.38,23.7151,61245,2000-01-01 17:00:45,3
61246,249,253,0.0029,0.0000,100.0,518.67,643.68,1607.72,1430.56,14.62,...,8.2525,0.03,395,2388,100.0,39.78,23.8270,61246,2000-01-01 17:00:46,2
61247,249,254,35.0046,0.8400,100.0,449.44,555.77,1381.29,1148.18,5.48,...,9.0515,0.02,337,2223,100.0,15.26,9.0774,61247,2000-01-01 17:00:47,1
61248,249,255,42.0030,0.8400,100.0,445.00,549.85,1369.75,1147.45,3.91,...,9.1207,0.02,333,2212,100.0,10.66,6.4341,61248,2000-01-01 17:00:48,0


In [3]:
# Here we use the given labels from test
# It seems the training problem requires creation of our own labels.
# Using these for now to get DFS set up
cutoff_times = data[['index', 'time']]
cutoff_times['label'] = data.pop('label')


In [4]:
def make_entityset(data):
    es = ft.EntitySet('Dataset')
    es.entity_from_dataframe(dataframe=data,
                             entity_id='recordings',
                             index='index',
                             time_index='time')

    es.normalize_entity(base_entity_id='recordings', 
                        new_entity_id='engines',
                        index='engine_no')

    es.normalize_entity(base_entity_id='recordings', 
                        new_entity_id='cycles',
                        index='time_in_cycles')
    return es
es = make_entityset(data)
es

Entityset: Dataset
  Entities:
    recordings (shape = [61249, 28])
    engines (shape = [249, 2])
    cycles (shape = [543, 2])
  Relationships:
    recordings.engine_no -> engines.engine_no
    recordings.time_in_cycles -> cycles.time_in_cycles

In [5]:
from featuretools.primitives import Sum, Mean, Std, Skew, Max, Min, Last, CumSum, Diff, Trend

cutoff_times2 = new_labels(data, labels)
fm, features = ft.dfs(entityset=es, 
                      target_entity='engines',
                      agg_primitives=[Max, Min, Last],
                      trans_primitives=[],
                      cutoff_time=cutoff_times2,
                      max_depth=3,
                      verbose=True)

Built 290 features
Elapsed: 04:00 | Remaining: 00:00 | Progress: 100%|██████████|| Calculated: 249/249 cutoff times


In [6]:
fm.tail()

,MAX(recordings.operational_setting_1),MAX(recordings.operational_setting_2),MAX(recordings.operational_setting_3),MAX(recordings.sensor_measurement_1),MAX(recordings.sensor_measurement_2),MAX(recordings.sensor_measurement_3),MAX(recordings.sensor_measurement_4),MAX(recordings.sensor_measurement_5),MAX(recordings.sensor_measurement_6),MAX(recordings.sensor_measurement_7),...,LAST(recordings.cycles.LAST(recordings.sensor_measurement_13)),LAST(recordings.cycles.LAST(recordings.sensor_measurement_14)),LAST(recordings.cycles.LAST(recordings.sensor_measurement_15)),LAST(recordings.cycles.LAST(recordings.sensor_measurement_16)),LAST(recordings.cycles.LAST(recordings.sensor_measurement_17)),LAST(recordings.cycles.LAST(recordings.sensor_measurement_18)),LAST(recordings.cycles.LAST(recordings.sensor_measurement_19)),LAST(recordings.cycles.LAST(recordings.sensor_measurement_20)),LAST(recordings.cycles.LAST(recordings.sensor_measurement_21)),label
engine_no,,,,,,,,,,,,,,,,,,,,,
245,42.0079,0.8414,100.0,518.67,642.44,1585.12,1398.14,14.62,21.61,555.05,...,2388.02,8079.85,9.3266,0.02,329,2212,100.0,10.57,6.3031,189
246,42.0078,0.8419,100.0,518.67,643.05,1594.00,1414.93,14.62,21.61,554.09,...,2388.21,8117.72,8.6757,0.03,369,2319,100.0,28.45,17.1816,98
247,42.0077,0.8420,100.0,518.67,642.60,1588.09,1408.18,14.62,21.58,557.52,...,2388.04,8143.43,8.3795,0.03,391,2388,100.0,39.26,23.4926,46
248,42.0076,0.8420,100.0,518.67,643.22,1597.83,1421.59,14.62,21.61,553.93,...,2388.16,8053.99,9.2722,0.03,367,2324,100.0,24.22,14.5655,40
249,42.0078,0.8420,100.0,518.67,642.79,1596.77,1410.40,14.62,21.58,560.09,...,2388.33,8084.90,9.2141,0.02,334,2223,100.0,14.97,8.9471,37


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_absolute_error

X = fm.copy().fillna(0)
y = X.pop('label')

def scoring_func(X, y, printing = True, feats=5):
    reg = RandomForestRegressor()
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    reg.fit(X_train, y_train)
    preds = reg.predict(X_test)
    scores = mean_absolute_error(preds, y_test)
    
    feature_imps = [(imp, X.columns[i]) for i, imp in enumerate(reg.feature_importances_)]
    feature_imps.sort()
    feature_imps.reverse()
    if printing:
        print('Mean Abs Error: {:.2f}'.format(scores))

        print('Feature Importances:')
        for i, f in enumerate(feature_imps[0:feats]):
            print('{}: {} [{:.3f}]'.format(i + 1, f[1], f[0]))
        print('-----\n')
    return preds, y_test, reg

preds, y_test, reg = scoring_func(X, y, feats=10)
print('Total unweighted error: {:.2f}'.format(
    np.sum([np.abs(y_test.values[i] - pred) for i, pred in enumerate(preds)])))

Mean Abs Error: 41.56
Feature Importances:
1: MAX(recordings.cycles.LAST(recordings.sensor_measurement_3)) [0.125]
2: LAST(recordings.time_in_cycles) [0.109]
3: MAX(recordings.cycles.LAST(recordings.sensor_measurement_11)) [0.068]
4: MAX(recordings.sensor_measurement_13) [0.053]
5: MAX(recordings.sensor_measurement_3) [0.046]
6: MAX(recordings.cycles.MAX(recordings.sensor_measurement_13)) [0.041]
7: MAX(recordings.cycles.MAX(recordings.sensor_measurement_17)) [0.041]
8: MAX(recordings.sensor_measurement_4) [0.030]
9: MAX(recordings.sensor_measurement_11) [0.027]
10: MAX(recordings.cycles.LAST(recordings.sensor_measurement_15)) [0.023]
-----

Total unweighted error: 2618.50


In [8]:
data2 = pd.read_csv('data/RUL_test.txt', sep=' ', header=-1, names=cols)
data2 = data2.drop(cols[-5:], axis=1)
data2['index'] = data2.index
data2.index = data2['index']
data2['time'] = pd.date_range('1/1/2000', periods=data2.shape[0], freq='s')

es2 = make_entityset(data2)
fm2 = ft.calculate_feature_matrix(entityset=es2, features=features, verbose=True)


Elapsed: 00:06 | Remaining: 00:00 | Progress: 100%|██████████|| Calculated: 1/1 cutoff times


In [11]:
X = fm2.copy().fillna(0)
y = pd.read_csv('data/RUL_test_truth.txt', sep=' ', header=-1, names=['label'], index_col=False)
preds2 = reg.predict(X)
print('Total unweighted error on Test: {:.2f}'.format(
    np.sum([np.abs(y.values[i] - pred) for i, pred in enumerate(preds2)])))

Total unweighted error on Test: 9396.50
